In [ ]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = 'https://www.imdb.com/chart/moviemeter'
driver.get(url)

In [ ]:
#creating a dictionary to store movie details
movies=dict()
movies["titles"] = []
movies["runtime"]= []
movies["rating"]=[]
movies["age_restriction"]=[]
movies["writers"]=[]
movies["directors"]=[]
movies["genre"]=[]
movies["trivia"]= []
#finding the element containing movie details
ad_grid = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul')
ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'sc-479faa3c-0.fMoWnh.cli-children')
#looping through each movie card
for i in ad_cards:
    try:
            #extracting movie title
            movies["titles"].append(i.find_element(By.CLASS_NAME, 'ipc-title.ipc-title--base.ipc-title--title.ipc-title-link-no-icon.ipc-title--on-textPrimary.sc-479faa3c-9.dkLVoC.cli-title').text)
            #open movie in new tab
            product_link_element = i.find_element(By.CLASS_NAME, 'ipc-title-link-wrapper')
            product_link_element.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)
            try:
                #extracting movie rating
                movies["rating"].append(driver.find_element(By.CLASS_NAME, 'sc-bde20123-1.cMEQkK').text) 
            except(NoSuchElementException,StaleElementReferenceException) as e :
                movies["rating"].append(0)
            try:
                #extracting movie age restriction
                movies["age_restriction"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]/a').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                movies["age_restriction"].append('none')
                
            try:
                #extracting movie runtime
                movies["runtime"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                movies["runtime"].append('0h 0m')
                
            #scrolling and extracting director details
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(2)
            try:
                movies["directors"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                try:
                    movies["directors"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div').text)
                except(NoSuchElementException,StaleElementReferenceException) as e :
                    movies["directors"].append('none')
                 
            
            try:
                #scrolling and extracting writer details
                movies["writers"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                try:
                    movies["writers"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[2]/div').text)
                except(NoSuchElementException,StaleElementReferenceException) as e :
                    movies["writers"].append('none')
                    
            driver.execute_script("window.scrollTo(500,3800);")
            time.sleep(4)
            try:
                # Scrolling and extracting genre details
                movies["genre"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[7]/div[2]/ul[2]/li[2]/div').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                movies["genre"].append('none')
                    
            try:
                # Scrolling and extracting trivia details
                movies["trivia"].append(driver.find_element(By.XPATH, '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[8]/div[2]/ul/li/div/div/div/div/div').text)
            except(NoSuchElementException,StaleElementReferenceException) as e :
                movies["trivia"].append('none')
            
           #closing the tab and switching back to the original tab
            time.sleep(2)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
    except(NoSuchElementException,StaleElementReferenceException) as e :
            print(e)
            #convert to dataframe and save in csv
    df = pd.DataFrame(movies)
    df.to_csv('Q2_data.csv')
   

In [ ]:
df=pd.read_csv("Q2_data.csv")
df

In [ ]:
import random

#getting user input for movie preferences
genre=input('Enter genre you want to watch: ')
rating=float(input('Enter minimum rating of movie you want to watch: '))
runtime=int(input('Enter maximum runtime of movie you want to watch: '))
age=input('Enter age restriction of movie  you want to watch: ')
length = len(df)

#looping through each row in the df
for i in range(length):
    #extracting hours and minutes from runtime
    hrs = int(df.loc[i, 'runtime'].split('h')[0])
    mins = int(df.loc[i, 'runtime'].split('h')[1].replace('m', ''))
    #calculating total runtime in minutes and updating the df
    df.loc[i, 'runtime(in minutes)'] = hrs * 60 + mins

#filtering the df based on user preferences
df2=df[(df['genre']==genre)&((df['rating'].astype(float))>=rating)&(df['age_restriction']==age)&(df['runtime(in minutes)']<=runtime)]

#checking if the df2 is empty
if len(df2)==0:
    #if yes then selecting a random row from da og df
    rand=random.randint(0,length-1)
    print("No movie matches ur requirement so selecting at random ")
    print(df.iloc[rand])
else:
    #if not printing df2
    print(df2)